<a href="https://colab.research.google.com/github/ishank296/aws_automation_scripts/blob/main/AWS01_S3_STS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Boto3 library:A python package to call aws apis**

In [1]:
!pip install boto3

     |████████████████████████████████| 132 kB 5.0 MB/s 
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 8.7 MB 44.6 MB/s 
     |████████████████████████████████| 138 kB 44.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


**Accessing any AWS service by creating client object**

In [2]:
import boto3
s3 = boto3.client('s3')

In [3]:
s3.waiter_names

['bucket_exists', 'bucket_not_exists', 'object_exists', 'object_not_exists']

In [ ]:
dir(s3)

['_PY_TO_OP_NAME',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_client_config',
 '_convert_to_request_dict',
 '_emit_api_params',
 '_endpoint',
 '_exceptions',
 '_exceptions_factory',
 '_get_waiter_config',
 '_load_exceptions',
 '_loader',
 '_make_api_call',
 '_make_request',
 '_register_handlers',
 '_request_signer',
 '_response_parser',
 '_serializer',
 '_service_model',
 'abort_multipart_upload',
 'can_paginate',
 'complete_multipart_upload',
 'copy',
 'copy_object',
 'create_bucket',
 'create_multipart_upload',
 'delete_bucket',
 'delete_bucket_analytics_configuration',
 'delete_bucket_cors',
 'delete_bucket_encryption',
 'delete_buc

In [ ]:
! touch ./aws/config

**Configure Credentials by setting Environment Variables**

In [35]:
from configparser import ConfigParser
import os
conf = ConfigParser()
conf.read("./aws/config")
os.environ['AWS_ACCESS_KEY_ID']=conf['default']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=conf['default']['AWS_SECRET_ACCESS_KEY']

**Accessing any service for which current user doesn't have access to gives an error as below**

In [36]:
s3 = boto3.client('s3')
s3.list_buckets()

ClientError: ignored

**AWS STS to assume role temporarily to access required service**

Make sure the current user is added in Trusted entities of the Role to be assumed in IAM

In [37]:
sts_client = boto3.client('sts')
sts_user=sts_client.assume_role(RoleArn='arn:aws:iam::130003587303:role/user_s3_tmp_role',
                       RoleSessionName='s3_demo')

In [38]:
new_session = boto3.Session(aws_access_key_id=sts_user['Credentials']['AccessKeyId'],
aws_secret_access_key=sts_user['Credentials']['SecretAccessKey'],
aws_session_token=sts_user['Credentials']['SessionToken'])

AWS S3 get bucket list

In [39]:
s3 = new_session.client('s3')
s3.list_buckets()

{'Buckets': [{'CreationDate': datetime.datetime(2021, 8, 3, 5, 38, 21, tzinfo=tzlocal()),
   'Name': 'aws-bkp-bkt'},
  {'CreationDate': datetime.datetime(2021, 4, 29, 14, 13, 15, tzinfo=tzlocal()),
   'Name': 'capgmn'},
  {'CreationDate': datetime.datetime(2021, 6, 21, 6, 35, 35, tzinfo=tzlocal()),
   'Name': 'concentrix-docs'},
  {'CreationDate': datetime.datetime(2022, 5, 14, 8, 22, 8, tzinfo=tzlocal()),
   'Name': 'demo095321'},
  {'CreationDate': datetime.datetime(2021, 5, 9, 9, 12, 57, tzinfo=tzlocal()),
   'Name': 'emr-data-input'},
  {'CreationDate': datetime.datetime(2021, 3, 2, 11, 7, 24, tzinfo=tzlocal()),
   'Name': 'rss-feed-v1'},
  {'CreationDate': datetime.datetime(2021, 4, 6, 15, 50, 54, tzinfo=tzlocal()),
   'Name': 'rule0transformer'},
  {'CreationDate': datetime.datetime(2021, 2, 7, 14, 54, 48, tzinfo=tzlocal()),
   'Name': 's3-asia-mumbai-bucket-000001'},
  {'CreationDate': datetime.datetime(2021, 6, 18, 15, 37, 29, tzinfo=tzlocal()),
   'Name': 'samaa'}],
 'Owner': 

AWS S3 list particular Bucket Objects

In [40]:
s3.list_objects(Bucket='emr-data-input')

{'Contents': [{'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'Key': 'csv-data/',
   'LastModified': datetime.datetime(2021, 5, 9, 9, 13, 37, tzinfo=tzlocal()),
   'Owner': {'DisplayName': 'ishank.kirty',
    'ID': '5ba4148975abfea24d6197daa5d9ff060d54b1fa57a9956ae87effd7f4c59848'},
   'Size': 0,
   'StorageClass': 'STANDARD'},
  {'ETag': '"e7a61945ed4541fc956a0e93859a712f"',
   'Key': 'csv-data/Heart.csv',
   'LastModified': datetime.datetime(2021, 5, 9, 12, 29, 59, tzinfo=tzlocal()),
   'Owner': {'DisplayName': 'ishank.kirty',
    'ID': '5ba4148975abfea24d6197daa5d9ff060d54b1fa57a9956ae87effd7f4c59848'},
   'Size': 17782,
   'StorageClass': 'STANDARD'},
  {'ETag': '"59997623160f97a9a3500233c9b66e06-8"',
   'Key': 'csv-data/household_power_consumption.txt',
   'LastModified': datetime.datetime(2021, 6, 13, 17, 0, 32, tzinfo=tzlocal()),
   'Owner': {'DisplayName': 'ishank.kirty',
    'ID': '5ba4148975abfea24d6197daa5d9ff060d54b1fa57a9956ae87effd7f4c59848'},
   'Size': 132960755,
   '

**AWS S3 Creating new bucket**

In [29]:
s3.create_bucket(Bucket='demo095321')

{'Location': '/demo095321',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Sat, 14 May 2022 08:22:08 GMT',
   'location': '/demo095321',
   'server': 'AmazonS3',
   'x-amz-id-2': 'QL6Ts48K8L0SEYVS1FsOYKaadMaz70wfNbNf9IcYI8I1tTQcGo+J3s7F4o9ZPSowWW6Xq0/3UUY=',
   'x-amz-request-id': '1JP31PYB1CC7F5WY'},
  'HTTPStatusCode': 200,
  'HostId': 'QL6Ts48K8L0SEYVS1FsOYKaadMaz70wfNbNf9IcYI8I1tTQcGo+J3s7F4o9ZPSowWW6Xq0/3UUY=',
  'RequestId': '1JP31PYB1CC7F5WY',
  'RetryAttempts': 0}}

**AWS S3 upload file in bucket**

In [31]:
s3.upload_file('./sample_data/mnist_train_small.csv','demo095321','mnist_small.csv')

**AWS S3 upload large files using multipart**



1.   create multipart upload - generate upload id
2.   upload part - upload actual data chunks with upload id and part number.
3.   complete multpart





In [46]:
file_path = './sample_data/mnist_train_small.csv'
chunk_size = 1024
def read_file_chunk(file_path,chunk_size):
  file_obj = open(file_path,'r')
  while True:
    data = file_obj.read(chunk_size)
    if not data:
      break
    yield data

In [56]:
multipart_upload=s3.create_multipart_upload(Bucket='demo095321',Key='mnist_test.csv')
part_list=list()
cnt=1
for part in read_file_chunk(file_path,chunk_size):
  try:
    resp=s3.upload_part(Body=part,
                 Bucket='demo095321',
                 Key='mnist_multipart.csv',
                 PartNumber=cnt,
                 UploadId=multipart_upload['UploadId'])
    uploaded_part={'ETag':resp['ETag'],'PartNumber':cnt}
    part_list.append(uploaded_part)
    cnt+=1
  except Exception as e:
    print(e)
    break
#s3.complete_multipart_upload(
#    Bucket='demo095321',
#    Key='mnist_multipart.csv',
#    UploadId=multipart_upload['UploadId'],
#    Parts=part_list
#)

An error occurred (NoSuchUpload) when calling the UploadPart operation: The specified upload does not exist. The upload ID may be invalid, or the upload may have been aborted or completed.
